### Original MDI Calculation

In [ ]:
//@version=5
indicator(title="Directional Movement Index", shorttitle="5_DMI", format=format.price, precision=4, timeframe="", timeframe_gaps=true)
lensig = input.int(14, title="ADX Smoothing", minval=1)
len = input.int(14, minval=1, title="DI Length")
up = ta.change(high)
down = -ta.change(low)
plusDM = na(up) ? na : (up > down and up > 0 ? up : 0)
minusDM = na(down) ? na : (down > up and down > 0 ? down : 0)
trur = ta.rma(ta.tr, len)
plus = fixnan(100 * ta.rma(plusDM, len) / trur)
minus = fixnan(100 * ta.rma(minusDM, len) / trur)
sum = plus + minus
adx = 100 * ta.rma(math.abs(plus - minus) / (sum == 0 ? 1 : sum), lensig)
plot(adx, color=#F50057, title="ADX")
plot(plus, color=#2962FF, title="+DI")
plot(minus, color=#FF6D00, title="-DI")

In [ ]:
//@version=5
strategy(title="Directional Movement Index with Strategy", shorttitle="5_DMI_Strategy")

// Input parameters
lensig = input.int(14, title="ADX Smoothing", minval=1)
len = input.int(14, minval=1, title="DI Length")

// Calculating ADX and DI+ / DI-
up = ta.change(high)
down = -ta.change(low)
plusDM = na(up) ? na : (up > down and up > 0 ? up : 0)
minusDM = na(down) ? na : (down > up and down > 0 ? down : 0)
trur = ta.rma(ta.tr, len)
plus = fixnan(100 * ta.rma(plusDM, len) / trur)
minus = fixnan(100 * ta.rma(minusDM, len) / trur)
sum = plus + minus
adx = 100 * ta.rma(math.abs(plus - minus) / (sum == 0 ? 1 : sum), lensig)

// Angle calculation for DI+ and DI-
angle_plus = math.atan(plus - plus[1])
angle_minus = math.atan(minus - minus[1])

// Threshold for a "high" angle
var_ADX_val = 18
var_minimum_bars = 5
var_high_angle_threshold = 0.2

// Buy/Sell conditions
buy_condition = (adx > var_ADX_val) and (plus > minus) and (plus[1] <= minus[1]) and (angle_plus > var_high_angle_threshold)
sell_condition = (adx > var_ADX_val) and (minus > plus) and (minus[1] <= plus[1]) and (angle_minus > var_high_angle_threshold)

// Cross condition while position is open
cross_condition = ta.crossover(minus, plus) or ta.crossover(plus, minus)

// Persistent bar counters
var int bars_since_buy = na
var int bars_since_sell = na

// Reset counters on position entry
if (buy_condition and strategy.position_size == 0)
    bars_since_buy := 0
if (sell_condition and strategy.position_size == 0)
    bars_since_sell := 0

// Increment counters if in position
if (strategy.position_size > 0)
    bars_since_buy := na(bars_since_buy) ? na : bars_since_buy + 1
if (strategy.position_size < 0)
    bars_since_sell := na(bars_since_sell) ? na : bars_since_sell + 1

// Plotting ADX and DI
plot(adx, color=color.new(#F50057, 0), title="ADX")
plot(plus, color=color.new(#2962FF, 0), title="+DI")
plot(minus, color=color.new(#FF6D00, 0), title="-DI")

// Strategy entries
if (buy_condition and strategy.position_size == 0)
    strategy.entry("Buy", strategy.long)
if (sell_condition and strategy.position_size == 0)
    strategy.entry("Sell", strategy.short)

// Close positions after 5 bars or on cross condition
if (strategy.position_size > 0 and (bars_since_buy >= var_minimum_bars or cross_condition))
    strategy.close("Buy")
if (strategy.position_size < 0 and (bars_since_sell >= var_minimum_bars or cross_condition))
    strategy.close("Sell")
